Для встановлення всіх залеежностей необхідно виконати команду, що запписана в наступному блоці ↓

In [ ]:
%pip install -r requirements.txt

In [2]:
import requests
from bs4 import BeautifulSoup
import re

Для початку формуємо список посилань на сторінки авторів.

In [3]:
URL = 'https://quotes.toscrape.com'

all_links = []

response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
quotes = soup.select('ul[class=pager] a')
url_list = [URL]
sufix = re.search(r'href="(/page/\d+/)"', str(quotes))
url = URL + sufix.group(1)
url_list.append(url)

while True:

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('ul[class=pager] li[class=next] a')
    sufix_a = re.search(r'href="(/page/\d+/)"', str(items))
    if not sufix_a:
        break
    url = URL + sufix_a.group(1)
    url_list.append(url)

In [5]:

def get_url_list(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    quotes = soup.select('div[class=quote] span a')
    
    for quote in quotes:
        sufix = re.search(r'href="(/author/[^"]+)"', str(quote))
        url = URL + sufix.group(1)
        all_links.append(url)
    return all_links

Оскільки один автор може зустрічатися кілька разів на сторінках з цитатами, його посилання може повторюватися. Щоб прибрати дублікати, перетворюємо список посилань у словник, де ключем є саме посилання (рядок). Завдяки унікальності ключів у словнику дублікати автоматично видаляються. Потім перетворюємо словник назад на список. Такий підхід дозволяє зберегти порядок елементів (на відміну від перетворення через set()).

In [6]:
for link in url_list:
    get_url_list(link)

mylist = list(dict.fromkeys(all_links))

In [ ]:
author = {}
authors = []
for link in mylist:

    html_doc = requests.get(link)
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    content = soup.select_one('div.author-details h3.author-title')
    fullname = content.get_text(strip=True)
    content = soup.select_one('div.author-details p span.author-born-date')
    born_date = content.get_text(strip=True)
    content = soup.select_one('div.author-details p span.author-born-location')
    born_location = content.get_text(strip=True)
    content = soup.select_one('div.author-details div.author-description')
    description = content.get_text(strip=True)
    author = {
        "fullname": fullname,
        "born_date": born_date,
        "born_location": born_location,
        "description": description
    }

    authors.append(author)

Після формування списка authors, записуємо його вміст в json файл.

In [8]:
import json

with open('authors.json', 'w', encoding='utf=8') as f:
    json.dump(authors, f, ensure_ascii=False, indent=4)

Підʼєднуємось до ДБ і призначаємо змінні для кожної колекції, в яку будемо записувати дані.

In [9]:
from pymongo import MongoClient
from getpass import getpass
uri = getpass("Enter Mongo URI: ")
client = MongoClient(uri)
db = client["goit-ds-hw-03"]
authors_db = db["authors"]
qoutes_db = db["qoutes"]

Читаємо записаний json файл і по елементно заносимо у відповідну колекцію.

In [11]:
with open('authors.json', 'r', encoding='utf=8') as f:
    data = json.load(f)
    for item in data:
        authors_db.insert_one(item)

Знаходимо і додаємо в словник необхідну інформацію для колекції qoutes.

In [12]:
qoutes = []
for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('div[class=quote]')
    pr_dickt = {}
    for item in items:
        tag = item.select_one('meta.keywords')
        tags = re.search(r'content="([^"]+)"', str(item))
        if tags:
            tags = tags.group(1).split(',')
        else:
            tags = None
        author = item.select_one('small.author').text
        quote = item.select_one('span.text').text
    
        pr_dickt = {
        "tags": tags,
        "author": author,
        "quote": quote
        }
        qoutes.append(pr_dickt)

In [13]:
with open('qoutes.json', 'w', encoding='utf=8') as f:
    json.dump(qoutes, f, ensure_ascii=False, indent=4)

In [14]:
with open('qoutes.json', 'r', encoding='utf=8') as f:
    data = json.load(f)
    for item in data:
        qoutes_db.insert_one(item)